# 测试集图像分类预测结果

使用训练好的图像分类模型，预测测试集的所有图像，得到预测结果表格。

同济子豪兄：https://space.bilibili.com/1900783

[代码运行云GPU环境](https://featurize.cn/?s=d7ce99f842414bfcaea5662a97581bd1)：GPU RTX 3060、CUDA v11.2

## 导入工具包

In [2]:
import os
from tqdm import tqdm

import numpy as np
import pandas as pd

from PIL import Image

import torch
import torch.nn.functional as F

# 有 GPU 就用 GPU，没有就用 CPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('device', device)

device cuda:0


## 图像预处理

In [4]:
from torchvision import transforms

# # 训练集图像预处理：缩放裁剪、图像增强、转 Tensor、归一化
# train_transform = transforms.Compose([transforms.RandomResizedCrop(224),
#                                       transforms.RandomHorizontalFlip(),
#                                       transforms.ToTensor(),
#                                       transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
#                                      ])

# 测试集图像预处理-RCTN：缩放、裁剪、转 Tensor、归一化
test_transform = transforms.Compose([transforms.Resize(256),
                                     transforms.CenterCrop(224),
                                     transforms.ToTensor(),
                                     transforms.Normalize(
                                         mean=[0.485, 0.456, 0.406], 
                                         std=[0.229, 0.224, 0.225])
                                    ])

## 载入测试集（和训练代码教程相同）

In [6]:
# 数据集文件夹路径
dataset_dir = 'fruit30_split'
test_path = os.path.join(dataset_dir, 'val')
from torchvision import datasets
# 载入测试集
test_dataset = datasets.ImageFolder(test_path, test_transform)
print('测试集图像数量', len(test_dataset))
print('类别个数', len(test_dataset.classes))
print('各类别名称', test_dataset.classes)
# 载入类别名称 和 ID索引号 的映射字典
idx_to_labels = np.load('idx_to_labels.npy', allow_pickle=True).item()
# 获得类别名称
classes = list(idx_to_labels.values())
print(classes)

测试集图像数量 1079
类别个数 30
各类别名称 ['哈密瓜', '圣女果', '山竹', '杨梅', '柚子', '柠檬', '桂圆', '梨', '椰子', '榴莲', '火龙果', '猕猴桃', '石榴', '砂糖橘', '胡萝卜', '脐橙', '芒果', '苦瓜', '苹果-红', '苹果-青', '草莓', '荔枝', '菠萝', '葡萄-白', '葡萄-红', '西瓜', '西红柿', '车厘子', '香蕉', '黄瓜']
['哈密瓜', '圣女果', '山竹', '杨梅', '柚子', '柠檬', '桂圆', '梨', '椰子', '榴莲', '火龙果', '猕猴桃', '石榴', '砂糖橘', '胡萝卜', '脐橙', '芒果', '苦瓜', '苹果-红', '苹果-青', '草莓', '荔枝', '菠萝', '葡萄-白', '葡萄-红', '西瓜', '西红柿', '车厘子', '香蕉', '黄瓜']


## 导入训练好的模型

In [7]:
model = torch.load('checkpoint/fruit30_pytorch_20220814.pth')
model = model.eval().to(device)

## 表格A-测试集图像路径及标注

In [8]:
test_dataset.imgs[:10]

[('fruit30_split/val/哈密瓜/106.jpg', 0),
 ('fruit30_split/val/哈密瓜/109.jpg', 0),
 ('fruit30_split/val/哈密瓜/114.jpg', 0),
 ('fruit30_split/val/哈密瓜/116.jpg', 0),
 ('fruit30_split/val/哈密瓜/118.png', 0),
 ('fruit30_split/val/哈密瓜/123.jpg', 0),
 ('fruit30_split/val/哈密瓜/127.jpg', 0),
 ('fruit30_split/val/哈密瓜/129.jpg', 0),
 ('fruit30_split/val/哈密瓜/131.jpg', 0),
 ('fruit30_split/val/哈密瓜/133.jpg', 0)]

In [9]:
img_paths = [each[0] for each in test_dataset.imgs]

In [10]:
df = pd.DataFrame()
df['图像路径'] = img_paths
df['标注类别ID'] = test_dataset.targets
df['标注类别名称'] = [idx_to_labels[ID] for ID in test_dataset.targets]

In [11]:
df

,图像路径,标注类别ID,标注类别名称
0,fruit30_split/val/哈密瓜/106.jpg,0,哈密瓜
1,fruit30_split/val/哈密瓜/109.jpg,0,哈密瓜
2,fruit30_split/val/哈密瓜/114.jpg,0,哈密瓜
3,fruit30_split/val/哈密瓜/116.jpg,0,哈密瓜
4,fruit30_split/val/哈密瓜/118.png,0,哈密瓜
...,...,...,...
1074,fruit30_split/val/黄瓜/87.jpg,29,黄瓜
1075,fruit30_split/val/黄瓜/9.jpg,29,黄瓜
1076,fruit30_split/val/黄瓜/91.png,29,黄瓜
1077,fruit30_split/val/黄瓜/94.jpg,29,黄瓜


## 表格B-测试集每张图像的图像分类预测结果，以及各类别置信度

In [12]:
# 记录 top-n 预测结果
n = 3

In [13]:
df_pred = pd.DataFrame()
for idx, row in tqdm(df.iterrows()):
    img_path = row['图像路径']
    img_pil = Image.open(img_path).convert('RGB')
    input_img = test_transform(img_pil).unsqueeze(0).to(device) # 预处理
    pred_logits = model(input_img) # 执行前向预测，得到所有类别的 logit 预测分数
    pred_softmax = F.softmax(pred_logits, dim=1) # 对 logit 分数做 softmax 运算

    pred_dict = {}

    top_n = torch.topk(pred_softmax, n) # 取置信度最大的 n 个结果
    pred_ids = top_n[1].cpu().detach().numpy().squeeze() # 解析出类别
    
    # top-n 预测结果
    for i in range(1, n+1):
        pred_dict['top-{}-预测ID'.format(i)] = pred_ids[i-1]
        pred_dict['top-{}-预测名称'.format(i)] = idx_to_labels[pred_ids[i-1]]
    pred_dict['top-n预测正确'] = row['标注类别ID'] in pred_ids
    # 每个类别的预测置信度
    for idx, each in enumerate(classes):
        pred_dict['{}-预测置信度'.format(each)] = pred_softmax[0][idx].cpu().detach().numpy()
        
    df_pred = df_pred.append(pred_dict, ignore_index=True)

1079it [00:27, 39.03it/s]


In [14]:
df_pred

,top-1-预测ID,top-1-预测名称,top-2-预测ID,top-2-预测名称,top-3-预测ID,top-3-预测名称,top-n预测正确,哈密瓜-预测置信度,圣女果-预测置信度,山竹-预测置信度,...,草莓-预测置信度,荔枝-预测置信度,菠萝-预测置信度,葡萄-白-预测置信度,葡萄-红-预测置信度,西瓜-预测置信度,西红柿-预测置信度,车厘子-预测置信度,香蕉-预测置信度,黄瓜-预测置信度
0,4.0,柚子,5.0,柠檬,7.0,梨,0.0,0.0021773854,4.5620004e-06,3.494981e-07,...,1.8101746e-07,1.2204301e-06,3.0817034e-06,9.9496065e-06,5.6826793e-06,0.00011113256,0.00012942357,3.994173e-07,4.3882874e-06,5.8307955e-07
1,6.0,桂圆,0.0,哈密瓜,8.0,椰子,1.0,0.3593502,2.473891e-07,6.390156e-06,...,8.4601425e-08,1.4647853e-06,1.0809676e-06,0.0014814788,4.5216493e-05,0.00017460674,7.725556e-07,3.1716188e-06,3.3343702e-05,2.5591053e-06
2,0.0,哈密瓜,26.0,西红柿,23.0,葡萄-白,1.0,0.5173351,0.0063806796,0.006869943,...,0.009829248,0.0076868637,0.0011496291,0.04022951,0.035186533,0.0015504733,0.27610925,0.00016955177,0.0060843485,0.0012194655
3,0.0,哈密瓜,16.0,芒果,4.0,柚子,1.0,0.7236518,6.884952e-05,1.7966098e-06,...,4.4178436e-05,0.00024698902,7.062981e-05,0.0014550196,2.550516e-06,0.0004602581,0.00083583395,2.1689974e-07,0.022085803,0.00041234458
4,4.0,柚子,11.0,猕猴桃,23.0,葡萄-白,0.0,0.0054146717,0.00013871821,4.1262163e-05,...,0.0007725807,7.507991e-05,8.939808e-05,0.09169763,0.00065894565,0.00046256272,0.0037965607,3.0871806e-08,0.00030570058,0.0005176165
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1074,29.0,黄瓜,25.0,西瓜,17.0,苦瓜,1.0,4.963991e-06,9.086888e-09,8.060354e-09,...,2.3248214e-08,2.4042232e-10,1.7068807e-09,1.5159178e-05,3.700708e-10,0.0007653143,5.824778e-07,3.4204437e-09,2.0657181e-07,0.9991691
1075,29.0,黄瓜,17.0,苦瓜,19.0,苹果-青,1.0,1.1492255e-07,1.241225e-08,1.0418997e-09,...,9.9390995e-08,3.520831e-09,9.062655e-09,3.7056296e-07,2.155812e-08,1.8662606e-06,6.1228455e-08,2.0431346e-08,7.6042267e-07,0.9997714
1076,29.0,黄瓜,17.0,苦瓜,23.0,葡萄-白,1.0,0.0009234467,0.00011089834,2.649959e-06,...,0.00027754836,0.00058677193,0.00010020598,0.0056848787,0.00045146354,1.2713363e-05,0.0001357052,5.936634e-07,0.0003697713,0.9494445
1077,29.0,黄瓜,17.0,苦瓜,10.0,火龙果,1.0,1.0736128e-06,1.5448064e-05,1.8266871e-07,...,0.0004145323,1.7989717e-06,9.843592e-05,0.00089812215,0.00033032725,0.0012592284,4.585372e-05,3.9792403e-06,9.1825284e-05,0.9849866


## 拼接AB两张表格

In [15]:
df = pd.concat([df, df_pred], axis=1)

In [16]:
df

,图像路径,标注类别ID,标注类别名称,top-1-预测ID,top-1-预测名称,top-2-预测ID,top-2-预测名称,top-3-预测ID,top-3-预测名称,top-n预测正确,...,草莓-预测置信度,荔枝-预测置信度,菠萝-预测置信度,葡萄-白-预测置信度,葡萄-红-预测置信度,西瓜-预测置信度,西红柿-预测置信度,车厘子-预测置信度,香蕉-预测置信度,黄瓜-预测置信度
0,fruit30_split/val/哈密瓜/106.jpg,0,哈密瓜,4.0,柚子,5.0,柠檬,7.0,梨,0.0,...,1.8101746e-07,1.2204301e-06,3.0817034e-06,9.9496065e-06,5.6826793e-06,0.00011113256,0.00012942357,3.994173e-07,4.3882874e-06,5.8307955e-07
1,fruit30_split/val/哈密瓜/109.jpg,0,哈密瓜,6.0,桂圆,0.0,哈密瓜,8.0,椰子,1.0,...,8.4601425e-08,1.4647853e-06,1.0809676e-06,0.0014814788,4.5216493e-05,0.00017460674,7.725556e-07,3.1716188e-06,3.3343702e-05,2.5591053e-06
2,fruit30_split/val/哈密瓜/114.jpg,0,哈密瓜,0.0,哈密瓜,26.0,西红柿,23.0,葡萄-白,1.0,...,0.009829248,0.0076868637,0.0011496291,0.04022951,0.035186533,0.0015504733,0.27610925,0.00016955177,0.0060843485,0.0012194655
3,fruit30_split/val/哈密瓜/116.jpg,0,哈密瓜,0.0,哈密瓜,16.0,芒果,4.0,柚子,1.0,...,4.4178436e-05,0.00024698902,7.062981e-05,0.0014550196,2.550516e-06,0.0004602581,0.00083583395,2.1689974e-07,0.022085803,0.00041234458
4,fruit30_split/val/哈密瓜/118.png,0,哈密瓜,4.0,柚子,11.0,猕猴桃,23.0,葡萄-白,0.0,...,0.0007725807,7.507991e-05,8.939808e-05,0.09169763,0.00065894565,0.00046256272,0.0037965607,3.0871806e-08,0.00030570058,0.0005176165
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1074,fruit30_split/val/黄瓜/87.jpg,29,黄瓜,29.0,黄瓜,25.0,西瓜,17.0,苦瓜,1.0,...,2.3248214e-08,2.4042232e-10,1.7068807e-09,1.5159178e-05,3.700708e-10,0.0007653143,5.824778e-07,3.4204437e-09,2.0657181e-07,0.9991691
1075,fruit30_split/val/黄瓜/9.jpg,29,黄瓜,29.0,黄瓜,17.0,苦瓜,19.0,苹果-青,1.0,...,9.9390995e-08,3.520831e-09,9.062655e-09,3.7056296e-07,2.155812e-08,1.8662606e-06,6.1228455e-08,2.0431346e-08,7.6042267e-07,0.9997714
1076,fruit30_split/val/黄瓜/91.png,29,黄瓜,29.0,黄瓜,17.0,苦瓜,23.0,葡萄-白,1.0,...,0.00027754836,0.00058677193,0.00010020598,0.0056848787,0.00045146354,1.2713363e-05,0.0001357052,5.936634e-07,0.0003697713,0.9494445
1077,fruit30_split/val/黄瓜/94.jpg,29,黄瓜,29.0,黄瓜,17.0,苦瓜,10.0,火龙果,1.0,...,0.0004145323,1.7989717e-06,9.843592e-05,0.00089812215,0.00033032725,0.0012592284,4.585372e-05,3.9792403e-06,9.1825284e-05,0.9849866


## 导出完整表格

In [17]:
df.to_csv('测试集预测结果.csv', index=False)